# Computing centralities from OSM data


Install and update `cityseer` if necessary.


In [1]:
# !pip install --upgrade cityseer

## Data and Extents

This example will make use of OSM data downloaded from the [OSM API](https://wiki.openstreetmap.org/wiki/API) for a 5000m radius around London Soho.

For additional information on network preparation, see the corresponding discussion on [graph cleaning](https://cityseer.benchmarkurbanism.com/guide#graph-cleaning).


In [2]:
from shapely import geometry

from cityseer.tools import graphs, io

# download from OSM
lng, lat = -0.13396079424572427, 51.51371088849723
buffer = 5000
plot_buffer = 3500
# creates a WGS shapely polygon
poly_wgs, poly_utm, _utm_zone_number, _utm_zone_letter = io.buffered_point_poly(lng, lat, buffer)
plot_bbox = poly_utm.centroid.buffer(plot_buffer).bounds

INFO:cityseer.tools.io:UTM conversion info: UTM zone number: 30, UTM zone letter: U


## Load

We can now load the OSM dataset and convert it to a format that can be used by cityseer for downstream calculations.


In [3]:
# generate OSM graph from polygon
G_utm = io.osm_graph_from_poly(poly_wgs, simplify=True, remove_parallel=True, iron_edges=False)
# decompose for higher resolution analysis
G_decomp = graphs.nx_decompose(G_utm, 25)
# prepare the data structures
nodes_gdf, _edges_gdf, network_structure = io.network_structure_from_nx(G_decomp, crs=32629)

INFO:cityseer.tools.io:Converting networkX graph from EPSG code 4326 to EPSG code 32630.
INFO:cityseer.tools.io:Processing node x, y coordinates.
100%|██████████| 152862/152862 [00:00<00:00, 384486.73it/s]
INFO:cityseer.tools.io:Processing edge geom coordinates, if present.
100%|██████████| 166303/166303 [00:00<00:00, 1003627.80it/s]
INFO:cityseer.tools.graphs:Generating interpolated edge geometries.
100%|██████████| 166303/166303 [00:04<00:00, 34650.65it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 152862/152862 [00:19<00:00, 7864.75it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████| 43118/43118 [00:00<00:00, 302099.41it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 40276/40276 [00:00<00:00, 63502.28it/s]
INFO:cityseer.tools.util:Creating nodes STR tree
100%|██████████| 38252/38252 [00:00<00:00, 114750.28it/s]
INFO:cityseer.tools.graphs:Consolidating nodes.
100%|██████████| 38252/38252 [00:16<00:00, 2298.59it/s]
I

In [4]:
# generate OSM graph from polygon
G_utm_messy = io.osm_graph_from_poly(poly_wgs, simplify=False)
G_utm_messy = graphs.nx_simple_geoms(G_utm_messy)
G_utm_messy = graphs.nx_remove_filler_nodes(G_utm_messy)
G_utm_messy = graphs.nx_remove_dangling_nodes(G_utm_messy, despine=15)
# decompose for higher resolution analysis
G_decomp_messy = graphs.nx_decompose(G_utm_messy, 25)
# generate dissolved weights
G_dissolved_wts = graphs.nx_locally_dissolve_edges(G_decomp_messy, 20)
# prepare the data structures
nodes_gdf_messy, _edges_gdf_messy, network_structure_messy = io.network_structure_from_nx(G_dissolved_wts, crs=32629)

INFO:cityseer.tools.io:Converting networkX graph from EPSG code 4326 to EPSG code 32630.
INFO:cityseer.tools.io:Processing node x, y coordinates.
100%|██████████| 152862/152862 [00:00<00:00, 431499.29it/s]
INFO:cityseer.tools.io:Processing edge geom coordinates, if present.
100%|██████████| 166303/166303 [00:00<00:00, 965670.90it/s]
INFO:cityseer.tools.graphs:Generating interpolated edge geometries.
100%|██████████| 166303/166303 [00:05<00:00, 29905.84it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 152862/152862 [00:21<00:00, 6967.67it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████| 43118/43118 [00:00<00:00, 274132.28it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 40276/40276 [00:00<00:00, 64221.50it/s]
INFO:cityseer.tools.graphs:Decomposing graph to maximum edge lengths of 25.
100%|██████████| 53462/53462 [00:25<00:00, 2107.47it/s]
INFO:cityseer.tools.graphs:Generating node weights based on locally dissolved edg

## Calculate centralities

The centrality methods can now be computed.


In [5]:
from cityseer.metrics import networks

distances = [250, 500, 2000]
# each of the below invocations will take around 5-6 minutes each depending on the available resources
# if computing wider area centralities, e.g. 20km, then use less decomposition to speed up the computation
nodes_gdf = networks.node_centrality_shortest(
    network_structure=network_structure,
    nodes_gdf=nodes_gdf,
    distances=distances,
)
# messy version with jitter and dissolved weights
nodes_gdf_messy = networks.node_centrality_shortest(
    network_structure=network_structure_messy,
    nodes_gdf=nodes_gdf_messy,
    distances=distances,
    jitter_scale=20,
)

  2%|▏         | 1555/94508 [00:12<12:52, 120.30it/s]

## Plots

Let's plot a selection of distance thresholds for each of the computed measures.


In [ ]:
import matplotlib.pyplot as plt
from cityseer import rustalgos
from cityseer.tools import plot

bg_colour = "#111"
distances = [250, 500, 2000]
betas = rustalgos.betas_from_distances(distances)
avg_dists = rustalgos.avg_distances_for_betas(betas)
for d, b, avg_d in zip(distances, betas, avg_dists):
    print(
        f"""
    "Gravity" index / spatial impedance weighted (closeness-like) centrality:
    Avg walking tolerance: {avg_d:.2f}m
    Beta: {b:.3f} (spatial impedance factor)
    Max walking tolerance: {d:.1f}m
    """
    )
    fig, axes = plt.subplots(1, 2, figsize=(10, 5), dpi=200, facecolor=bg_colour)
    plot.plot_scatter(
        axes[0],
        network_structure.node_xs,
        network_structure.node_ys,
        nodes_gdf[f"cc_metric_node_beta_{d}"],
        bbox_extents=plot_bbox,
        cmap_key="magma",
        face_colour=bg_colour,
    )
    plot.plot_scatter(
        axes[1],
        network_structure_messy.node_xs,
        network_structure_messy.node_ys,
        nodes_gdf_messy[f"cc_metric_node_beta_{d}"],
        bbox_extents=plot_bbox,
        cmap_key="magma",
        face_colour=bg_colour,
    )
    plt.show()

for d, b, avg_d in zip(distances, betas, avg_dists):
    print(
        f"""
    Spatial impedance weighted betweenness centrality:
    Avg walking tolerance: {avg_d:.2f}m
    Beta: {b:.3f} (spatial impedance factor)
    Max walking tolerance: {d:.1f}m
    """
    )
    fig, axes = plt.subplots(1, 2, figsize=(10, 5), dpi=200, facecolor=bg_colour)
    plot.plot_scatter(
        axes[0],
        network_structure.node_xs,
        network_structure.node_ys,
        nodes_gdf[f"cc_metric_node_betweenness_{d}"],
        bbox_extents=plot_bbox,
        cmap_key="magma",
        s_max=2,
        face_colour=bg_colour,
    )
    plot.plot_scatter(
        axes[1],
        network_structure_messy.node_xs,
        network_structure_messy.node_ys,
        nodes_gdf_messy[f"cc_metric_node_betweenness_{d}"],
        bbox_extents=plot_bbox,
        cmap_key="magma",
        s_max=2,
        face_colour=bg_colour,
    )
    plt.show()